In [2]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 10.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# ======== Fuzzy Logic System ========

# Variabel input
temperature = ctrl.Antecedent(np.arange(15, 41, 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'humidity')

# Variabel output
fan_speed = ctrl.Consequent(np.arange(0, 101, 1), 'fan_speed')

# Fungsi keanggotaan untuk suhu
temperature['low'] = fuzz.trimf(temperature.universe, [15, 15, 24])
temperature['normal'] = fuzz.trimf(temperature.universe, [22, 29, 40])
temperature['high'] = fuzz.trimf(temperature.universe, [30, 36, 41])

# Fungsi keanggotaan untuk kelembaban
humidity['normal'] = fuzz.trimf(humidity.universe, [20, 20, 35])
humidity['high'] = fuzz.trimf(humidity.universe, [30, 50, 100])
humidity['very_high'] = fuzz.trimf(humidity.universe, [50, 80, 100])

# Fungsi keanggotaan untuk kecepatan kipas
fan_speed['fast'] = fuzz.trimf(fan_speed.universe, [0, 0, 25])
fan_speed['medium'] = fuzz.trimf(fan_speed.universe, [20, 35, 50])
fan_speed['normal'] = fuzz.trimf(fan_speed.universe, [35, 50, 75])
fan_speed['slow'] = fuzz.trimf(fan_speed.universe, [50, 75, 100])

# Visualisasi fungsi keanggotaan
# temperature.view()
# humidity.view()
# fan_speed.view()

# Aturan fuzzy
rule1 = ctrl.Rule(temperature['high'] & humidity['normal'], fan_speed['fast'])
rule2 = ctrl.Rule(temperature['high'] & humidity['high'], fan_speed['medium'])
rule3 = ctrl.Rule(temperature['high'] & humidity['very_high'], fan_speed['normal'])
rule4 = ctrl.Rule(temperature['normal'] & humidity['normal'], fan_speed['slow'])
rule5 = ctrl.Rule(temperature['normal'] & humidity['high'], fan_speed['slow'])
rule6 = ctrl.Rule(temperature['low'] & humidity['normal'], fan_speed['slow'])
rule7 = ctrl.Rule(temperature['low'] & humidity['high'], fan_speed['slow'])

# Sistem kontrol fuzzy
fan_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
fan = ctrl.ControlSystemSimulation(fan_control)

# Simulasi input baru sesuai soal
fan.input['temperature'] = 22  # Ubah input suhu menjadi 22C
fan.input['humidity'] = 65     # Kelembaban tetap
fan.compute()

# Output
print(f"Kecepatan kipas yang disarankan (suhu 22°C, kelembaban 65%): {fan.output['fan_speed']:.2f}")
# fan_speed.view(sim=fan)

# Penjelasan:
# Suhu 22°C berada di daerah transisi fuzzy set 'low' dan 'normal'. Oleh karena itu, memiliki keanggotaan parsial di keduanya.
# Ini mencerminkan fleksibilitas fuzzy logic dalam menangani ambiguitas.

# ======== Certainty Factor Section ========

def hitung_cf_total(gejala_dict):
    cf_total = 0
    for cf in gejala_dict.values():
        cf_total = cf_total + cf * (1 - cf_total)
    return cf_total

# Gejala awal
gejala = {
    "batuk": 0.7,
    "pilek": 0.6,
    "meriang": 0.8,
    "sakit_kepala": 0.7,
    "nyeri_sendi": 0.6
}

cf_awal = hitung_cf_total(gejala)
print("\n===== CF Awal =====")
print("CF Total (awal):", round(cf_awal, 4))

# Ubah nilai CF 'meriang' menjadi 0.2
gejala["meriang"] = 0.2
cf_setelah = hitung_cf_total(gejala)
print("\n===== CF Setelah Perubahan 'meriang' =====")
print("CF Total (setelah perubahan):", round(cf_setelah, 4))

# Tambah 5 gejala baru
gejala.update({
    "mual": 0.5,
    "lelah": 0.6,
    "nyeri_otot": 0.4,
    "demam": 0.2,
    "sesak_nafas": 0.7
})
cf_dengan_tambahan = hitung_cf_total(gejala)
print("\n===== CF Setelah Tambahan Gejala =====")
print("CF Total (dengan gejala tambahan):", round(cf_dengan_tambahan, 4))

# Penjelasan:
# - Nilai CF 'meriang' diubah dari 0.8 ke 0.2, sehingga kontribusinya ke CF total menurun.
# - Penambahan 5 gejala baru meningkatkan keyakinan sistem (CF total naik), karena semakin banyak indikasi penyakit.
# - CF memperlihatkan kombinasi keyakinan berdasarkan kehadiran beberapa gejala.


Kecepatan kipas yang disarankan (suhu 22°C, kelembaban 65%): 75.00

===== CF Awal =====
CF Total (awal): 0.9971

===== CF Setelah Perubahan 'meriang' =====
CF Total (setelah perubahan): 0.9885

===== CF Setelah Tambahan Gejala =====
CF Total (dengan gejala tambahan): 0.9997
